In [ ]:
# default_exp scrapper

# Tweet Scrapper

> Use Twitter API to get tweets

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
#%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#export
import pandas as pd
import tweepy
from os import getenv
from dotenv import load_dotenv
import time
import datetime


In [ ]:
#hide
# Load .env only in Notebook, it will be populated at runtime by docker
from pathlib import Path
env_path = Path('..') / '.env'
if env_path.is_file():
    load_dotenv(dotenv_path=env_path)
else:
    raise "No .env found !"

True

In [ ]:
#export
def get_users_accounts(csv = None):
    csv = getenv("USERS_CSV") if csv is None else csv
    return pd.read_csv(csv)

In [ ]:
#hide
df_users = get_users_accounts(USERS_CSV)

In [ ]:
#hide
df_users.head(2)

,twitter,nom,nom_de_famille,prenom,sexe,twitter_tweets,twitter_followers,twitter_following,twitter_listed,twitter_favourites,twitter_verified,twitter_protected,twitter_id,twitter_name,twitter_description,twitter_created_at,sites_web,url_institution,slug,url_nosdeputes_api
0,damienabad,Damien Abad,Abad,Damien,H,6457,23043,6427,0,4116,True,False,76584619,Damien Abad,Président des députés #LR @republicains_AN • D...,2009-09-23T07:21:37,http://www.damienabad.fr|http://www.damien-aba...,http://www2.assemblee-nationale.fr/deputes/fic...,damien-abad,https://www.nosdeputes.fr/damien-abad/csv
1,AbadieCaroline,Caroline Abadie,Abadie,Caroline,F,1747,5712,2305,127,1220,True,False,507168683,Abadie Caroline,Députée de l'Isère et coprd Groupe d’étude #Pr...,2012-02-28T10:05:23,http://www.abadiecaroline.fr|https://twitter.c...,http://www2.assemblee-nationale.fr/deputes/fic...,caroline-abadie,https://www.nosdeputes.fr/caroline-abadie/csv


In [ ]:
#hide
user_id = df_users.twitter_id[0]

In [ ]:
#export
# Convert UTC to Local based on the current date (do not work well around the day we change from/to summertime)
def datetime_from_utc_to_local(utc_datetime):
    now_timestamp = time.time()
    offset = datetime.datetime.fromtimestamp(now_timestamp) - datetime.datetime.utcfromtimestamp(now_timestamp)
    return utc_datetime + offset

In [ ]:
#export
   
def get_user_tweets(user_id):
    auth = tweepy.AppAuthHandler(getenv("TWITTER_CONSUMER_KEY"), getenv("TWITTER_CONSUMER_SECRET"))
    api = tweepy.API(auth)
    tweets = []
    for tweet in tweepy.Cursor(api.user_timeline, id=user_id, tweet_mode='extended').items(100):
        #print(tweet)
        tweet = {
            'user_id':user_id,
            'tweet_id':tweet.id,
            'datetime_utc':tweet.created_at,
            'datetime_local':datetime_from_utc_to_local(tweet.created_at),
            'text':tweet.full_text, # .encode('utf8')
            'retweet':tweet.retweet_count,
            'favorite':tweet.favorite_count
        }
        tweets.append(tweet)
    return tweets

In [ ]:
#hide
tweets = get_user_tweets(user_id)

In [ ]:
#hide
tweets[0]['datetime_utc']
created_date_local = datetime_from_utc_to_local(tweets[0]['datetime_utc'])
created_date_local

datetime.datetime(2021, 3, 18, 18, 50, 8)

datetime.datetime(2021, 3, 18, 19, 50, 8)

In [ ]:
#hide
df_tweets = pd.DataFrame(tweets)

In [ ]:
#hide
pd.set_option('display.max_colwidth', None)
df_tweets.head(7)

,user_id,tweet_id,datetime_utc,datetime_local,text,retweet,favorite
0,76584619,1372621356223885322,2021-03-18 18:50:08,2021-03-18 19:50:08,"Quand la vaccination est à l'arrêt, le confinement est en marche. Terrible constat d'échec que d'être contraint de confiner une nouvelle fois. \nIl est temps de sortir de cette gestion de crise sanitaire à la fois anxiogène, bavarde et contradictoire. #Castex19h #confinement3",16,64
1,76584619,1372618033701748736,2021-03-18 18:36:55,2021-03-18 19:36:55,"Cette décision de reconfinement est la conséquence directe d'une politique de vaccination chaotique. Avec la suspension puis la reprise du vaccin Astrazeneca, le Gouvernement invente le stop and go vaccinal. Vaccinez et augmentez nos lits de réa, sans quoi on devra tjrs confiner",41,137
2,76584619,1372292566469730307,2021-03-17 21:03:38,2021-03-17 22:03:38,Retour terrain : plusieurs centaines de rendez-vous annulés pour la primo-vaccination #AstraZeneca . Conséquence directe de la suspension de ce vaccin par le gouvernement. La défiance est en marche.,13,29
3,76584619,1372241502412419074,2021-03-17 17:40:43,2021-03-17 18:40:43,« Je ne crois pas à la primaire (…) @xavierbertrand est le rempart face au Rassemblement national (…) il rassemble bien au-delà de la droite » #EuropeSoir #Europe1 #confinement,93,189
4,76584619,1372205745631072257,2021-03-17 15:18:38,2021-03-17 16:18:38,"🎙Retrouvons-nous ce soir à 18h20 sur @Europe1 pour l’émission #EuropeSoir, où je serai l’invité politique de @JulianBugier \n- #LR @lesRepublicains https://t.co/1cqSp0r3rG",2,12
5,76584619,1372177146924265473,2021-03-17 13:25:00,2021-03-17 14:25:00,"""Je suis opposé au #confinement le week-end, qui est une demi-mesure, sans efficacité sanitaire. Soit on peut tenir avec le système actuel de renfort sanitaire ; soit on ne peut pas tenir et on assume un reconfinement, en maintenant les écoles ouvertes» #MidiNews @CNEWS #LR https://t.co/bjN3XV5Uh3",4,16
6,76584619,1372159719645806595,2021-03-17 12:15:45,2021-03-17 13:15:45,"""Je ne comprends pas le timing de la suspension d'#AstraZenaca : pourquoi 24h avant l'avis de l’OMS et 72h avant l’Agence Européenne du Médicament ? Avant la suspension 43% des Français faisaient confiance en ce vaccin, maintenant c’est 20%. On a redonné du pouvoir aux Antivax!"" https://t.co/ve2ejPmUfJ",30,65


In [ ]:
#hide
df_tweets.to_csv('tweets-sample.csv')

In [ ]:
#export
'''
Get all the tweets of all the users
input : a list of twitter_id
output : a list of all tweets
'''
def get_all_tweet(users_id, logger = None):
    tweets = []
    total_users = len(users_id)
    for i, user_id in enumerate(users_id):
        tweets += (get_user_tweets(user_id))
        if i % 10 == 0:
            if logger is not None:
                logger.debug(f'Processing user {i} / {total_users} - {(i*100//total_users*100)/100}%')
        #if i>3:
        #    break
    return tweets

In [ ]:
#hide
tweets = get_all_tweet()
len(tweets)

  0%|          | 3/603 [00:07<23:28,  2.35s/it]


400

In [ ]:
#hide
tweets[3]
i=20
total_users=603
print((i*100//total_users*100)/100)

{'user_id': 76584619,
 'tweet_id': 1372241502412419074,
 'datetime_utc': datetime.datetime(2021, 3, 17, 17, 40, 43),
 'datetime_local': datetime.datetime(2021, 3, 17, 18, 40, 43),
 'text': '« Je ne crois pas à la primaire (…) @xavierbertrand est le rempart face au Rassemblement national (…) il rassemble bien au-delà de la droite » #EuropeSoir #Europe1 #confinement',
 'retweet': 93,
 'favorite': 189}

3.0
